In [1]:
# You can also solve it in Python using sqlite3 and pandas

import sqlite3

connection = sqlite3.connect('breakout.db')

cursor = connection.cursor()

In [2]:
cursor.execute('DROP TABLE IF EXISTS parents')

In [3]:
# Create the table
sql_command = '''
CREATE TABLE parents AS
  SELECT "abraham" AS parent, "barack" AS child UNION
  SELECT "abraham",           "clinton"         UNION
  SELECT "delano",            "herbert"         UNION
  SELECT "fillmore",          "abraham"         UNION
  SELECT "fillmore",          "delano"          UNION
  SELECT "fillmore",          "grover"          UNION
  SELECT "eisenhower",        "fillmore";
'''

cursor.execute(sql_command)

In [4]:
connection.commit()

# Q1 Simple SELECTS (on the parents table)
* 1: SELECT all records in the table.
* 2: SELECT child and parent, where abraham is the parent.
* 3: SELECT all children that have an 'e' in their name (hint: use LIKE and '%e%').
* 4: SELECT all unique parents (use SELECT DISTINCT) and order them by name, descending order (i.e. fillmore first)
* 5: **Difficult***: SELECT all dogs that are siblings (one-to-one relations). Only show a sibling pair once. To do this you need to select two times from the parents table.

In [6]:
import pandas as pd
import numpy as np
pd.read_sql_query('SELECT * FROM parents;', connection)

,parent,child
0,abraham,barack
1,abraham,clinton
2,delano,herbert
3,eisenhower,fillmore
4,fillmore,abraham
5,fillmore,delano
6,fillmore,grover


In [7]:
df = pd.read_sql_query('SELECT parent, child '
                       'FROM parents '
                       'WHERE parent="abraham" '
                        , connection)
df

,parent,child
0,abraham,barack
1,abraham,clinton


In [8]:
df = pd.read_sql_query('SELECT child '
                       'FROM parents '
                       'WHERE child LIKE "%e%" '
                        , connection)
df

,child
0,herbert
1,fillmore
2,delano
3,grover


In [9]:
df = pd.read_sql_query('SELECT DISTINCT parent '
                       'FROM parents '
                        'ORDER BY parent DESC', connection)
df

,parent
0,fillmore
1,eisenhower
2,delano
3,abraham


In [10]:
df = pd.read_sql_query('SELECT a.child as first, b.child as second '
                       'FROM parents as a, parents as b '
                        'WHERE a.parent=b.parent and a.child<b.child', connection)
df

,first,second
0,barack,clinton
1,abraham,delano
2,abraham,grover
3,delano,grover


<div id="sec4"></div>

## Q2 Joins

Create a new table called dogs, which indicates the fur type of every dog. In the image above: long haired dogs = red dashed box, curly haired dogs = black fluffy box, and short haired dogs = grey dotted box. 

Create the table by running:
	
	CREATE TABLE dogs AS
	  SELECT "abraham" AS name, "long" AS fur UNION
	  SELECT "barack",          "short"       UNION
	  SELECT "clinton",         "long"        UNION
	  SELECT "delano",          "long"        UNION
	  SELECT "eisenhower",      "short"       UNION
	  SELECT "fillmore",        "curly"       UNION
	  SELECT "grover",          "short"       UNION
	  SELECT "herbert",         "curly";

1. COUNT the number of short haired dogs
2. JOIN tables parents and dogs and SELECT the parents of curly dogs.
2. **Difficult**: JOIN tables parents and dogs, and SELECT the parents and children that have the same fur type. Only show them once.

In [11]:
sql_command = '''
CREATE TABLE dogs AS
  SELECT "abraham" AS name, "long" AS fur UNION
  SELECT "barack",          "short"       UNION
  SELECT "clinton",         "long"        UNION
  SELECT "delano",          "long"        UNION
  SELECT "eisenhower",      "short"       UNION
  SELECT "fillmore",        "curly"       UNION
  SELECT "grover",          "short"       UNION
  SELECT "herbert",         "curly";
'''
cursor.execute(sql_command)

In [12]:
connection.commit()

In [13]:
# 1.
pd.read_sql_query('SELECT Count(*) FROM dogs WHERE fur="short"',connection)

,Count(*)
0,3


In [16]:
sql_command='''
SELECT parents.parent as parents_of_curly_dogs
FROM dogs
JOIN parents ON parents.child=dogs.name
WHERE fur="curly";
'''
pd.read_sql_query(sql_command,connection)

,parents_of_curly_dogs
0,eisenhower
1,delano


In [18]:
sql_command='''
SELECT parents.parent, c.fur, parents.child, d.fur
FROM parents, dogs as c, dogs as d
WHERE parent=c.name and child=d.name and c.fur=d.fur;
'''
pd.read_sql_query(sql_command,connection)

,parent,fur,child,fur
0,abraham,long,clinton,long



## Q3 Aggregate functions, numerical logic and grouping

Create a new table with many different animals. The table includes the animal's kind, number of legs and weight. Create it by running:

	create table animals as
	 select "dog" as kind, 4 as legs, 20 as weight union
	 select "cat" , 4 , 10 union
	 select "ferret" , 4 , 10 union
	 select "parrot" , 2 , 6 union
	 select "penguin" , 2 , 10 union
	select "t-rex" , 2 , 12000;
	
1. SELECT the animal with the minimum weight. Display kind and min_weight.
2. Use aggregate function AVG to display a table with the average number of legs and the average weight.
3. SELECT the animal kind(s) that have more than two legs, but weighs less than 20. Display kind, weight, legs.
4. SELECT the average weight for all the animals with 2 legs and the animals with 4 legs (by using GROUP BY).

.

In [19]:
sql_command = '''
create table animals as
 select "dog" as kind, 4 as legs, 20 as weight union
 select "cat" , 4 , 10 union
 select "ferret" , 4 , 10 union
 select "parrot" , 2 , 6 union
 select "penguin" , 2 , 10 union
select "t-rex" , 2 , 12000;
'''
cursor.execute(sql_command)
connection.commit()

In [20]:
# 1.
sql_command = '''
SELECT kind, MIN(weight)
FROM animals;
'''
pd.read_sql_query(sql_command,connection)

,kind,MIN(weight)
0,parrot,6


In [21]:
sql_command='''
SELECT AVG(legs), AVG(weight)
FROM animals;
'''
pd.read_sql_query(sql_command,connection)

,AVG(legs),AVG(weight)
0,3.0,2009.333333


In [22]:
sql_command='''
SELECT kind, legs, weight
FROM animals
WHERE legs>2 and weight<20;
'''
pd.read_sql_query(sql_command,connection)

,kind,legs,weight
0,cat,4,10
1,ferret,4,10


In [23]:
sql_command='''
SELECT legs, AVG(weight)
FROM animals
GROUP BY legs;
'''
pd.read_sql_query(sql_command,connection)

,legs,AVG(weight)
0,2,4005.333333
1,4,13.333333


In [24]:
connection.commit() 

connection.close()